In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [54]:
#read the dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [55]:
#Preprocess the data drop unnecessary columns
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [56]:
#Encode categorical variables
label_encoder=LabelEncoder()
data['Gender']=label_encoder.fit_transform(data['Gender'])

In [57]:
#once hot encode geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [58]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [59]:
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [60]:
#drop the original geography column
data.drop(['Geography'],axis=1,inplace=True)
#concatenate the new columns with the original dataframe
data=pd.concat([data,geo_encoded_df],axis=1)

In [61]:
#Save the encoders and scaler for future use
with open('label_encoder.pkl','wb') as f:
    pickle.dump(label_encoder,f)

with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)

In [ ]:
#Divide the dataset into independent and dependent features
y=data['Exited']
X=data.drop(['Exited'],axis=1)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [64]:
#split into train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
#Scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [65]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [66]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [67]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [69]:
X_train.shape[1]

12

In [70]:
##build ann model

model=Sequential([
Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
Dense(32,activation='relu'),
Dense(1,activation='sigmoid')
])

In [71]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [74]:
## Setup tensorboard
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

log_dir='logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [75]:
#Setup early stopping
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [76]:
### training the model

history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,early_stopping])

Epoch 1/100


250/250 [==============================] - 7s 5ms/step - loss: 0.4785 - accuracy: 0.7818 - val_loss: 0.4010 - val_accuracy: 0.8310
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3869 - accuracy: 0.8431 - val_loss: 0.3565 - val_accuracy: 0.8585
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3516 - accuracy: 0.8564 - val_loss: 0.3488 - val_accuracy: 0.8580
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3419 - accuracy: 0.8600 - val_loss: 0.3442 - val_accuracy: 0.8615
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3357 - accuracy: 0.8615 - val_loss: 0.3447 - val_accuracy: 0.8575
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3322 - accuracy: 0.8620 - val_loss: 0.3422 - val_accuracy: 0.8640
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3300 - accuracy: 0.8629 - val_loss: 0.3420 - val_accuracy: 0.86

In [77]:
model.save('model.h5')

c:\Users\sreevathsava.pak\OneDrive - TIGER ANALYTICS\Documents\project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [78]:
# load tensorboard extension

%load_ext tensorboard

In [81]:
%tensorboard --logdir logs/fit 

Reusing TensorBoard on port 6006 (pid 18484), started 0:02:23 ago. (Use '!kill 18484' to kill it.)

In [ ]:
# load the pickle files

